In [1]:
import networkx as nx
import parse
import utils
import sys
from os.path import basename, normpath
import glob

In [2]:
def Sort_Tuple(tup):  
  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    return(sorted(tup, key = lambda x: x[2]))

In [3]:
def list_to_dict(l):
    result_dict = dict()
    for t in l:
        first = t[0]
        second = t[1]
        value = t[2]
        if first not in result_dict.keys():
            result_dict[first] = dict()
        result_dict[first][second] = value  
    return result_dict

In [4]:
def merge_Node(graph, i, j):

    for k in graph.adj[j]:
        if k == i:
            continue
        graph[i][k]['happiness'] +=  graph[j][k]['happiness']
        graph[i][k]["stress"] +=  graph[j][k]["stress"]

    graph.remove_node(j)

In [5]:
def processData(result):
    l = result.keys()
    l = list(l)
    for i in range(len(l)):
        result[i] = result[l[i]]
    removed = []
    for key in result:
        if key >= len(l):
            removed += [key]
    for key in removed:
        del result[key]
    return result

In [6]:
def processData2(dictionary):
    result = dict()
    for key in dictionary:
        result[key] = dictionary[key]['Member']
    return result

In [7]:
def solve2(G, G2, s):
    backup = dict()
    for i in range(len(G.nodes)):
        backup[i] = {'Member': [i], 'happiness': 0, 'stress': 0}
    while True:
        happiness_list = []
        for (u, v, wt) in G.edges.data('happiness'):
            happiness_list += [(u, v, wt)]
        stress_list = []
        for (u, v, wt) in G.edges.data('stress'):
            stress_list += [(u, v, wt)]
        new_happiness_list = Sort_Tuple(happiness_list)
        new_happiness_list.reverse()
        new_stress_list = Sort_Tuple(stress_list)
        stress_dict = list_to_dict(new_stress_list)
        returned = True
        for i in range(len(new_stress_list)):
            temp_tuple = new_stress_list[i]
            first = temp_tuple[0]
            second = temp_tuple[1]
            if G[first][second]["stress"] + backup[first]['stress'] + backup[second]['stress'] <= s / (len(G.nodes) - 1):
                backup[first]['Member'] = backup[second]['Member'] + backup[first]['Member']
                backup[first]['happiness'] = backup[second]['happiness'] + backup[first]['happiness'] + G[first][second]['happiness']
                backup[first]['stress'] = backup[second]['stress'] + backup[first]['stress'] + G[first][second]['stress']
                merge_Node(G, first, second)
                del backup[second]
                returned = False
                break
        if returned:
            result = processData(backup)
            room_assigned = processData2(result)
            output = utils.convert_dictionary(room_assigned)
            return output, len(backup), utils.calculate_happiness(output, G2)
        if (len(G.nodes) == 1):
            result = processData(backup)
            room_assigned = processData2(result)
            output = utils.convert_dictionary(room_assigned)
            print(utils.calculate_happiness(output, G2))
            return output, len(backup), utils.calculate_happiness(output, G2)

            
        
    
    

In [8]:
def solve3(G, G2, s):
    backup = dict()
    for i in range(len(G.nodes)):
        backup[i] = {'Member': [i], 'happiness': 0, 'stress': 0}
    while True:
        happiness_list = []
        for (u, v, wt) in G.edges.data('happiness'):
            happiness_list += [(u, v, wt)]
        stress_list = []
        for (u, v, wt) in G.edges.data('stress'):
            stress_list += [(u, v, wt)]
        new_happiness_list = Sort_Tuple(happiness_list)
        new_happiness_list.reverse()
        stress_dict = list_to_dict(stress_list)
        returned = True
        for i in range(len(new_happiness_list)):
            temp_tuple = new_happiness_list[i]
            first = temp_tuple[0]
            second = temp_tuple[1]
            if G[first][second]["stress"] + backup[first]['stress'] + backup[second]['stress'] <= s / (len(G.nodes) - 1):
                backup[first]['Member'] = backup[second]['Member'] + backup[first]['Member']
                backup[first]['happiness'] = backup[second]['happiness'] + backup[first]['happiness'] + G[first][second]['happiness']
                backup[first]['stress'] = backup[second]['stress'] + backup[first]['stress'] + G[first][second]['stress']
                merge_Node(G, first, second)
                del backup[second]
                returned = False
                break
        if returned:
            result = processData(backup)
            room_assigned = processData2(result)
            output = utils.convert_dictionary(room_assigned)
            return output, max(output.values()) + 1, utils.calculate_happiness(output, G2)
        if (len(G.nodes) == 1):
            result = processData(backup)
            room_assigned = processData2(result)
            output = utils.convert_dictionary(room_assigned)
            return output, max(output.values()) + 1, utils.calculate_happiness(output, G2)

In [9]:
def solve(G, s):
    backup = dict()
    number_of_students = len(G.nodes)
    for i in range(number_of_students):
        backup[i] = []
    for i in range(number_of_students):
        backup[0] += [i]
    while True:
        temp_result = utils.convert_dictionary(backup)
        temp_number_rooms = len(set(temp_result.values()))
        if utils.is_valid_solution(temp_result, G, s, temp_number_rooms):
            return temp_result, temp_number_rooms, utils.calculate_happiness(temp_result, G)
        temp_most_stress = 0
        temp_most_stress_room = 0
        for i in range(number_of_students):
            if utils.calculate_stress_for_room(backup[i], G) > temp_most_stress:
                temp_most_stress = utils.calculate_stress_for_room(backup[i], G)
                temp_most_stress_room = i
        temp_most_happiness = 0
        temp_least_influenced_student = backup[temp_most_stress_room][0]
        for i in range(len(backup[temp_most_stress_room])):
            temp_backup = backup[temp_most_stress_room].copy()
            temp_backup.pop(i)
            if utils.calculate_happiness_for_room(temp_backup, G) > temp_most_happiness:
                temp_most_happiness = utils.calculate_happiness_for_room(temp_backup, G)
                temp_least_influenced_student = backup[temp_most_stress_room][i]
        backup[temp_most_stress_room].remove(temp_least_influenced_student)
        for i in range(number_of_students):
            temp_backup_room = backup[i].copy()
            temp_number_room = temp_number_rooms
            if temp_backup_room == []:
                temp_number_room += 1
            temp_backup_room.append(temp_least_influenced_student)
            temp_stress_budget_per_room = s/temp_number_room
            if utils.calculate_stress_for_room(temp_backup_room, G) <= temp_stress_budget_per_room:
                backup[i].append(temp_least_influenced_student)
                break;

In [10]:
def solve5(G, s):
    backup = dict()
    number_of_students = len(G.nodes)
    for i in range(number_of_students):
        backup[i] = []
    for i in range(number_of_students):
        backup[0] += [i]
    while True:
        temp_result = utils.convert_dictionary(backup)
        temp_number_rooms = len(set(temp_result.values()))
        if utils.is_valid_solution(temp_result, G, s, temp_number_rooms):
            return temp_result, temp_number_rooms, utils.calculate_happiness(temp_result, G)
        temp_most_stress = 0
        temp_most_stress_room = 0
        for i in range(number_of_students):
            if utils.calculate_stress_for_room(backup[i], G) > temp_most_stress:
                temp_most_stress = utils.calculate_stress_for_room(backup[i], G)
                temp_most_stress_room = i
        temp_least_stress = 100000
        temp_most_influenced_student = backup[temp_most_stress_room][0]
        for i in range(len(backup[temp_most_stress_room])):
            temp_backup = backup[temp_most_stress_room].copy()
            temp_backup.pop(i)
            if utils.calculate_stress_for_room(temp_backup, G) < temp_least_stress:
                temp_least_stress = utils.calculate_stress_for_room(temp_backup, G)
                temp_most_influenced_student = backup[temp_most_stress_room][i]
        backup[temp_most_stress_room].remove(temp_most_influenced_student)
        for i in range(number_of_students):
            temp_backup_room = backup[i].copy()
            temp_number_room = temp_number_rooms
            if temp_backup_room == []:
                temp_number_room += 1
            temp_backup_room.append(temp_most_influenced_student)
            temp_stress_budget_per_room = s/temp_number_room
            if utils.calculate_stress_for_room(temp_backup_room, G) <= temp_stress_budget_per_room:
                backup[i].append(temp_most_influenced_student)
                break;

In [11]:
def solve6(G, s):
    backup = dict()
    number_of_students = len(G.nodes)
    for i in range(number_of_students):
        backup[i] = []
    for i in range(number_of_students):
        backup[0] += [i]
    while True:
        temp_result = utils.convert_dictionary(backup)
        temp_number_rooms = len(set(temp_result.values()))
        if utils.is_valid_solution(temp_result, G, s, temp_number_rooms):
            return temp_result, temp_number_rooms, utils.calculate_happiness(temp_result, G)
        temp_most_stress = 0
        temp_most_stress_room = 0
        for i in range(number_of_students):
            if utils.calculate_stress_for_room(backup[i], G) > temp_most_stress:
                temp_most_stress = utils.calculate_stress_for_room(backup[i], G)
                temp_most_stress_room = i
        temp_most_happiness = 0
        temp_least_influenced_student = backup[temp_most_stress_room][0]
        for i in range(len(backup[temp_most_stress_room])):
            temp_backup = backup[temp_most_stress_room].copy()
            temp_backup.pop(i)
            if utils.calculate_happiness_for_room(temp_backup, G) > temp_most_happiness:
                temp_most_happiness = utils.calculate_happiness_for_room(temp_backup, G)
                temp_least_influenced_student = backup[temp_most_stress_room][i]
        backup[temp_most_stress_room].remove(temp_least_influenced_student)
        temp_new_happiness = 0
        temp_assigned = number_of_students
        for i in range(number_of_students):
            temp_backup_room = backup[i].copy()
            temp_number_room = temp_number_rooms
            if temp_backup_room == []:
                temp_number_room += 1
            temp_backup_room.append(temp_least_influenced_student)
            temp_stress_budget_per_room = s/temp_number_room
            if utils.calculate_stress_for_room(temp_backup_room, G) <= temp_stress_budget_per_room:
                if utils.calculate_happiness_for_room(temp_backup_room, G) >= temp_new_happiness:
                    temp_new_happiness = utils.calculate_happiness_for_room(temp_backup_room, G)
                    temp_assigned = i
        if temp_assigned < number_of_students:
            backup[temp_assigned].append(temp_least_influenced_student)

In [12]:
def solve7(G, s):
    backup = dict()
    number_of_students = len(G.nodes)
    for i in range(number_of_students):
        backup[i] = []
    for i in range(number_of_students):
        backup[0] += [i]
    while True:
        temp_result = utils.convert_dictionary(backup)
        temp_number_rooms = len(set(temp_result.values()))
        if utils.is_valid_solution(temp_result, G, s, temp_number_rooms):
            return temp_result, temp_number_rooms, utils.calculate_happiness(temp_result, G)
        temp_most_stress = 0
        temp_most_stress_room = 0
        for i in range(number_of_students):
            if utils.calculate_stress_for_room(backup[i], G) > temp_most_stress:
                temp_most_stress = utils.calculate_stress_for_room(backup[i], G)
                temp_most_stress_room = i
        temp_least_stress = 100000000000000000000000000000000
        temp_most_influenced_student = backup[temp_most_stress_room][0]
        for i in range(len(backup[temp_most_stress_room])):
            temp_backup = backup[temp_most_stress_room].copy()
            temp_backup.pop(i)
            if utils.calculate_stress_for_room(temp_backup, G) < temp_least_stress:
                temp_least_stress = utils.calculate_stress_for_room(temp_backup, G)
                temp_most_influenced_student = backup[temp_most_stress_room][i]
        backup[temp_most_stress_room].remove(temp_most_influenced_student)
        temp_new_stress = 10000000000000000000000000000000000
        temp_assigned = number_of_students
        for i in range(number_of_students):
            temp_backup_room = backup[i].copy()
            temp_number_room = temp_number_rooms
            if temp_backup_room == []:
                temp_number_room += 1
            temp_backup_room.append(temp_most_influenced_student)
            temp_stress_budget_per_room = s/temp_number_room
            if utils.calculate_stress_for_room(temp_backup_room, G) <= temp_stress_budget_per_room:
                if utils.calculate_stress_for_room(temp_backup_room, G) <= temp_new_stress:
                    temp_new_stress = utils.calculate_stress_for_room(temp_backup_room, G)
                    temp_assigned = i
        if temp_assigned < number_of_students:
            backup[temp_assigned].append(temp_most_influenced_student)

In [18]:
import copy

def compress(intput):
    count = 0
    res = {}

    for i in intput:
        if intput[i] != []:
            res[count] = intput[i] 
            count += 1
    return res, count

def solve4(G, G2, s):  
    now_happyness = 0
    roomN = 4
    this_count = 10
    room_ = {}
    now_res = dict()
    for i in range(roomN):
        room_[i] = []
    for i in range(10):
        now_res[i] = [i]
    now_res = utils.convert_dictionary(now_res)
    for p1 in range(roomN): 
        for p2 in range(roomN):
            for p3 in range(roomN):
                for p4 in range(roomN):
                    for p5 in range(roomN):
                        for p6 in range(roomN):
                            for p7 in range(roomN):
                                for p8 in range(roomN):
                                    for p9 in range(roomN):
                                        for p10 in range(roomN):
                                            room_assigned = copy.deepcopy(room_)
                                            room_assigned[p1].append(0)
                                            room_assigned[p2].append(1)
                                            room_assigned[p3].append(2)
                                            room_assigned[p4].append(3)
                                            room_assigned[p5].append(4)
                                            room_assigned[p6].append(5)
                                            room_assigned[p7].append(6)
                                            room_assigned[p8].append(7)
                                            room_assigned[p9].append(8)
                                            room_assigned[p10].append(9)
                                            output = utils.convert_dictionary(room_assigned)
                                            if utils.is_valid_solution(output, G2, s, len(set(output.values()))):
                                                this_Happyness = utils.calculate_happiness(output, G)
                                                if this_Happyness > now_happyness:
                                                    now_res = output
                                                    now_happyness = this_Happyness
                                                    this_count = len(set(output.values()))
    
    
    #if this_count == 0:
        #room_assigned = {}
        #for i in range(10):
            #room_assigned[i] = [i]
        #output = utils.convert_dictionary(room_assigned)
        #this_count = 10
        #now_happyness = 0 
    print(0)
    return now_res, this_count

In [20]:
def solver(G, G2, G3, s):
    d1, k1, v1 = solve(G, s)
    d2, k2, v2 = solve2(G, G2, s)
    d3, k3, v3 = solve3(G3, G2, s)
    d5, k5, v5 = solve5(G2, s)
    d6, k6, v6 = solve6(G2, s)
    print(v1, v2, v3, v5, v6)
    v_max = max(v1, v2, v3, v5, v6)
    if (v1 == v_max):
        return d1, k1
    elif (v2 == v_max):
        return d2, k2
    elif (v3 == v_max):
        return d3, k3
    elif (v5 == v_max):
        return d5, k5
    else:
        return d6, k6

In [21]:
G, stress_budget = parse.read_input_file("inputs/small-1.in")
G2, stress_budget2 = parse.read_input_file("inputs/small-1.in")
G3, stress_budget2 = parse.read_input_file("inputs/small-1.in")

In [46]:
solver(G, G2, G3, stress_budget)

0
1
2
3
4
5
6
7
8
9
{0: 9, 1: 9, 2: 9, 3: 9, 4: 9, 5: 9, 6: 9, 7: 9, 8: 9, 9: 9}
744.03 881.6600000000001 720.31 677.3599999999999 963.0 744.03


({1: 0, 3: 0, 5: 0, 6: 0, 7: 0, 0: 1, 8: 1, 4: 2, 2: 2, 9: 2}, 3)

In [22]:
if __name__ == '__main__':
    inputs = glob.glob('inputs/*')
    for input_path in inputs:
        output_path = 'outputs/' + basename(normpath(input_path))[:-3] + '.out'
        G, s = parse.read_input_file(input_path)
        G2, s = parse.read_input_file(input_path)
        G3, s = parse.read_input_file(input_path)
        D, k = solver(G, G2, G3, s)
        assert utils.is_valid_solution(D, G2, s, k)
        happiness = utils.calculate_happiness(D, G2)
        parse.write_output_file(D, output_path)

2230.316 986.469 1097.6609999999998 966.364 2230.316
3729.0 3433.0 3512.0 3577.0 3729.0
224.682 990.586 990.586 533.747 593.4449999999999
159.95299999999997 174.07599999999996 267.903 89.178 90.316
237.986 252.834 252.834 229.651 237.986
77.0 82.15 109.5 133.33999999999997 77.0
1682.0 1085.0 1328.0 1610.0 1682.0
64.77 23.78 25.919999999999998 64.77000000000001 64.77
491.8979999999999 231.24 351.21000000000004 383.56 491.89799999999997
886.6239999999999 1131.156 886.308 1537.0720000000001 886.6239999999999
14949.0 9633.0 10005.0 14949.0 14949.0
72.553 81.24300000000001 131.23499999999999 126.04799999999999 73.01800000000001
138.0 155.0 195.0 460.0 138.0
32.9 16.8 45.6 33.2 32.9
1750.0480000000002 125.65299999999999 1477.879 197.69299999999998 1750.0479999999998
404.37999999999994 425.68 512.02 263.2799999999999 429.27
8.159999999999998 0.2 4.219999999999999 4.219999999999999 8.159999999999998
2252.21 1685.42 3157.36 3157.36 2252.21
671.221 517.9259999999999 879.796 661.1460000000001 671

1524.0220000000002 1251.3849999999998 1916.862 1775.1109999999999 1524.022
5725.6630000000005 3574.124 6683.938 5981.280999999999 5725.663
544.0 803.0 857.0 719.0 608.0
62.407 62.407 62.407 62.407 62.407
96.36 162.03 147.23000000000002 102.16 101.71000000000001
1008.2230000000001 476.45599999999996 882.576 940.49 1008.2230000000001
376.482 404.504 503.614 686.8489999999999 376.48199999999997
1107.562 4165.438 4165.438 1969.6110000000003 1468.639
90.0 90.0 90.0 367.0 90.0
2226.405 2229.508 3477.123 1746.8240000000003 2542.5029999999997
368.36 188.324 296.492 188.324 368.36
69.965 71.962 148.702 124.433 106.02600000000001
1773.85 674.4300000000001 903.7299999999998 732.0199999999999 1773.85
148.91000000000003 207.54000000000002 322.48 390.63000000000005 148.91000000000003
692.9720000000001 881.247 1136.4950000000001 1099.65 692.972
135.214 64.771 114.901 379.744 135.214
252.856 122.445 229.07399999999998 23.294 252.856
210.844 202.43699999999998 232.851 232.019 210.84400000000002
112.8 8

167.18999999999997 157.75 270.5700000000001 163.45000000000002 213.48999999999998
74.345 34.785000000000004 67.011 43.81999999999999 74.345
95.39599999999997 98.45599999999996 112.50599999999999 106.39799999999998 96.90700000000001
607.438 249.379 469.559 587.3159999999999 607.438
1304.5759999999998 1687.1570000000004 1899.1599999999999 1699.6139999999998 1644.5510000000002
130.046 129.51 162.846 162.473 130.046
64.424 24.448 38.564 39.614 64.424
370.49399999999997 391.073 423.63800000000003 598.0780000000001 368.612
22.0 6.0 20.0 15.0 22.0
156.878 344.894 388.16900000000004 344.894 156.878
555.0 120.0 270.0 280.0 555.0
259.34700000000004 316.21900000000005 320.88000000000005 316.219 254.39600000000002
56.82900000000001 0.0 0.0 31.40300000000001 56.138999999999996
48.0 33.0 48.0 72.0 48.0
346.59200000000004 584.929 643.664 328.42400000000004 345.12600000000003
3007.5339999999997 1172.97 3391.7019999999998 3630.2 3007.5339999999997
115.0 170.0 170.0 86.0 115.0
57.0 62.0 62.0 57.0 57.0
6

711.0 251.0 438.0 422.0 711.0
55.0 45.0 66.0 64.0 55.0
0.932 0.6250000000000001 0.755 0.978 0.932
105.0 81.0 67.0 84.0 105.0
85.74899999999998 70.957 134.07999999999996 109.596 89.84799999999998
94.534 85.842 63.971000000000004 70.426 94.53399999999999
3944.151 1783.574 3425.8370000000004 4030.018 3944.151
3537.536 4688.374000000001 6337.919999999999 3954.2299999999996 4179.996
33233.23100000001 33233.23100000001 33233.23100000001 33233.23100000001 33233.23100000001
176.35399999999998 253.12400000000002 258.594 342.512 431.5
3018.8439999999996 5245.961 5245.961 2693.66 3101.1620000000003
586.061 758.283 940.589 997.07 646.971
17.0 20.0 56.0 22.0 27.0
501.0769999999999 410.31100000000004 473.157 593.138 501.07699999999994
35704.0 33748.0 33816.0 34602.0 35704.0
19.310000000000002 8.139999999999999 13.6 13.22 19.310000000000002
87.0 75.0 132.0 75.0 81.0
1680.603 1155.4420000000002 1988.4679999999998 1382.779 1680.6030000000003
4623.7699999999995 6071.409 6243.814 5263.325 4660.251
63.0 6

1330.9650000000001 548.7489999999999 1023.8149999999998 1923.7220000000002 1342.641
774.091 1068.7469999999998 1214.2179999999998 1052.279 823.9050000000001
33.01599999999999 39.59 39.512 70.75599999999999 33.016000000000005
25.775000000000002 17.310999999999996 34.381 16.077 25.273
55.521 74.63399999999999 87.901 87.901 55.521
46.858 47.032000000000004 53.075 103.11899999999999 44.39
52.0 8.0 16.0 42.5 52.0
62.467 62.607 143.44500000000002 130.868 101.312
117.97000000000001 86.35400000000001 134.891 131.971 117.97
6132.05 5659.384 6301.365 11037.921999999999 6132.049999999999
675.2610000000001 657.0089999999999 820.6239999999999 803.014 675.2610000000001
18001.185 12511.326 12947.296 17937.885000000002 18001.185
106.99299999999998 58.099 73.219 104.044 106.99300000000001
73.0 49.0 95.0 88.0 73.0
1867.3739999999998 2048.7409999999995 2718.1819999999993 2369.728 1809.2300000000002
496.864 338.96999999999997 462.55199999999996 376.285 496.864
564.483 363.69100000000003 711.7830000000001 